<a href="https://colab.research.google.com/github/burraco135/Notebooks/blob/master/audio_course_music_genre_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a music genre classifier (Unit 4)

## Goal
Achieve 87% accuracy on this dataset

In [ ]:
%pip install -q datasets gradio transformers numpy huggingface_hub evaluate
%pip install -q -U accelerate

In [ ]:
from datasets import load_dataset

# load gtzan dataset
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

In [ ]:
# create validation set from the dataset
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

In [ ]:
# audio files are represented as 1-dimensional arrays
#   the value of the array represents the amplitude at that timestep
# the sampling rate is 22,050 Hz
#   there are 22,050 amplitude valllues sampled per second
gtzan["train"][0]

In [ ]:
# int2str can map the genre feature (int) to human-readable names (str)
id2label_fn = gtzan["train"].features["genre"].int2str
print("Genre: ", id2label_fn(gtzan["train"][0]["genre"]))

In [ ]:
# listen 4 random audio files from train with their labels
import gradio as gr

def generate_audio():
  example = gtzan["train"].shuffle()[0]
  audio = example["audio"]
  return (
      audio["sampling_rate"],
      audio["array"]
  ), id2label_fn(example["genre"])

with gr.Blocks() as demo:
  with gr.Column():
    for _ in range(4):
      audio, label = generate_audio()
      output = gr.Audio(audio, label=label)

demo.launch(debug=True)

In [ ]:
# transformers proves a class that can automatically select the correct feature
from transformers import AutoFeatureExtractor

model_id = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=False
)

In [ ]:
# feature extractor sampling rate is 16,000
sampling_rate = feature_extractor.sampling_rate
print("Feature extractor sampling rate:", sampling_rate)

In [ ]:
# resampling all audio file for feature extractor
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))
gtzan["train"][0]

In [ ]:
# rescaling each sample to zero mean and unit variance
import numpy as np

sample = gtzan["train"][0]["audio"]
print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

In [ ]:
# truncate audio longer than 30s
max_duration = 30.0

def preprocess_function(examples):
  audio_arrays = [x["array"] for x in examples["audio"]]
  inputs = feature_extractor(
      audio_arrays,
      sampling_rate=feature_extractor.sampling_rate,
      max_length=int(feature_extractor.sampling_rate * max_duration),
      truncation=True,
      return_attention_mask=True
  )
  return inputs

gtzan_encoded = gtzan.map(
    preprocess_function, remove_columns=["audio", "file"], batched=True, num_proc=1
)

In [ ]:
# rename genre column to label
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [ ]:
# obtain label mappings from dataset
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}

label2id = {v: k for k, v in id2label.items()}

print("Genre: ", id2label["7"])

In [ ]:
# fine-tuning a model
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

In [ ]:
# linking the notebook to Hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# define the training arguments
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 4
gradient_accumulation_steps = 4
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False
)

In [ ]:
# define the metrics
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  """Computes accuracy on a batch of predictions"""
  predictions = np.argmax(eval_pred.predictions, axis=1)
  return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
# train the model
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
from huggingface_hub import whoami

whoami()

In [ ]:
# push the model on the hub
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)